# Tutorial #3.1: Off-road Navigaiton with CL-RRT

This tutorial runs closed-loop RRT (CL-RRT) with probabilistic traversability model. Some studies exploit such a sampling-based global planning algorithm to tightly couple robot dynamics with planning. We implement them to compare its performance against other planning algorithms.

In [ ]:
%matplotlib inline
# Import the necessary libraries

# Add the parent directory to the path so that the environment can be found
import sys
import os

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))

# Import the necessary libraries
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

# Import the necessary classes
from src.environments.grid_map import GridMap
from src.simulator.planetary_env import PlanetaryEnv
from src.simulator.problem_formulation.utils import ModelConfig
from src.simulator.problem_formulation.robot_model import UnicycleModel
from src.simulator.problem_formulation.objectives import Objectives
from src.planners.global_planners.sampling_based.cl_rrt import CLRRT
from src.prediction_models.trainers.utils import ParamsModelTraining
from src.prediction_models.trainers.utils import load_model_state_dict
from src.prediction_models.trainers.utils import load_slip_regressors
from src.prediction_models.terrain_classifiers.unet import Unet
from src.prediction_models.slip_regressors.gpr import GPModel
from src.prediction_models.traversability_predictors.classifier_and_regressor import (
    TraversabilityPredictor,
)

sns.set()

device = "cuda" if torch.cuda.is_available() else "cpu"

1. Prepare ML models representing probabilistic traversability by loading both terrain classifier and slip regressors.

In [ ]:
dataset_index = 1
num_total_terrain_classes = 10

# Set the path to model directory
base_directory = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
model_directory = os.path.join(
    base_directory, f"trained_models/dataset{dataset_index:02d}"
)

# Set the model configuration to load the terrain classifier
params_model_training = ParamsModelTraining(
    batch_size=16,
    learning_rate=1e-3,
    weight_decay=0e00,
    num_epochs=100,
    save_interval=None,
    device=device,
)
model_directory_unet = os.path.join(
    model_directory,
    "Unet",
    f"bs{params_model_training.batch_size:03d}_"
    f"lr{params_model_training.learning_rate:.0e}_"
    f"wd{params_model_training.weight_decay:.0e}_"
    f"epochs{params_model_training.num_epochs:03d}",
    "models/best_model.pth",
)

# Load the terrain classifier
terrain_classifier = Unet(classes=num_total_terrain_classes).to(device)

# Load the trained model
terrain_classifier = load_model_state_dict(
    terrain_classifier, model_directory_unet, device
)

# Set the model configuration to load the slip regressor
params_model_training = ParamsModelTraining(
    learning_rate=1e-1, num_iterations=100, device=device
)

model_directory_gpr = os.path.join(
    model_directory,
    "GPR",
    f"lr{params_model_training.learning_rate:.0e}_"
    f"iters{params_model_training.num_iterations:03d}",
)

# Load the slip regressor
all_gp_models = load_slip_regressors(
    model=GPModel,
    num_terrain_classes=num_total_terrain_classes,
    model_directory=model_directory_gpr,
    train_data_directory=os.path.join(
        base_directory, f"datasets/dataset{dataset_index:02d}/"
    ),
    device=device,
)

# Integrate the terrain classifier and the slip regressor into the traversability predictor
traversability_predictor = TraversabilityPredictor(
    terrain_classifier=terrain_classifier,
    slip_regressors=all_gp_models,
    device=device,
)

2. Prepare data for traversability prediction and motion planning problems.

In [ ]:
subset_index = 1
instance_name = "000_000"
data_directory = os.path.join(
    base_directory,
    f"datasets/dataset{dataset_index:02d}/test/subset{subset_index:02d}/{instance_name}.pt",
)

# Load the data
data_item = torch.load(data_directory, map_location=device)
tensors = data_item["tensors"]
distributions = data_item["distributions"]

3. Predict traversability by providing color and slope information into the predictor.

In [ ]:
predictions = traversability_predictor.predict(
    colors=tensors["colors"], slopes=tensors["slopes"]
)
distributions["predictions"] = predictions

# Torch -> NumPy conversion
slips_mean = distributions["latent_models"].mean.cpu().numpy()
slips_stddev = distributions["latent_models"].stddev.cpu().numpy()
pred_mean = distributions["predictions"].mean.cpu().numpy()
pred_stddev = distributions["predictions"].stddev.cpu().numpy()

# Visualize the input color and slope images
fig, axs = plt.subplots(1, 2, figsize=(8, 4))

axs[0].imshow(tensors["colors"].permute(1, 2, 0).cpu().numpy())
axs[0].set_title("Color Map")
axs[0].axis("off")
axs[0].set_aspect("equal")

axs[1].imshow(tensors["t_classes"].cpu().numpy(), cmap="jet")
axs[1].set_title("Terrain Class Map")
axs[1].axis("off")
axs[1].set_aspect("equal")
plt.tight_layout()


fig, axs = plt.subplots(1, 2, figsize=(8, 4))

img = axs[0].imshow(tensors["heights"].cpu().numpy(), cmap="turbo")
axs[0].set_title("Height Map")
axs[0].axis("off")
axs[0].set_aspect("equal")
fig.colorbar(img, ax=axs[0], orientation="vertical", fraction=0.046, pad=0.04)

img = axs[1].imshow(tensors["slopes"].cpu().numpy(), cmap="turbo")
axs[1].set_title("Slope Map")
axs[1].axis("off")
axs[1].set_aspect("equal")
fig.colorbar(img, ax=axs[1], orientation="vertical", fraction=0.046, pad=0.04)
plt.tight_layout()


# Visualize the traversability predictions
fig, axs = plt.subplots(1, 2, figsize=(8, 4))
# mean map
mean_img = axs[0].imshow(slips_mean, cmap="turbo", vmin=0, vmax=1)
axs[0].set_title("Traversability Map - GT Mean")
axs[0].axis("off")
fig.colorbar(mean_img, ax=axs[0], orientation="vertical", fraction=0.046, pad=0.04)

# stddev map
stddev_img = axs[1].imshow(slips_stddev, cmap="turbo", vmin=0.1, vmax=0.2)
axs[1].set_title("Traversability Map - GT Stddev")
axs[1].axis("off")
fig.colorbar(stddev_img, ax=axs[1], orientation="vertical", fraction=0.046, pad=0.04)
plt.tight_layout()


fig, axs = plt.subplots(1, 2, figsize=(8, 4))
# mean map
mean_img = axs[0].imshow(pred_mean, cmap="turbo", vmin=0, vmax=1)
axs[0].set_title("Traversability Map - Predict. Mean")
axs[0].axis("off")
fig.colorbar(mean_img, ax=axs[0], orientation="vertical", fraction=0.046, pad=0.04)

# stddev map
stddev_img = axs[1].imshow(pred_stddev, cmap="turbo", vmin=0.1, vmax=0.2)
axs[1].set_title("Traversability Map - Predict. Stddev")
axs[1].axis("off")
fig.colorbar(stddev_img, ax=axs[1], orientation="vertical", fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

4. Construct the gym simulator by wrapping grid map.

In [ ]:
# Initialize the grid map
grid_map = GridMap(
    grid_size=64,
    resolution=0.5,
    tensors=tensors,
    distributions=distributions,
    instance_name=instance_name,
)

# Wrap the grid map into a planetary environment
delta_t = 0.1
time_limit = 100
start_pos = torch.tensor([8, 8], device=device)
goal_pos = torch.tensor([24, 24], device=device)
seed = 0
env = PlanetaryEnv(
    grid_map=grid_map,
    start_pos=start_pos,
    goal_pos=goal_pos,
    seed=seed,
    delta_t=delta_t,
    time_limit=time_limit,
    stuck_threshold=0.3,
    render_mode="rgb_array",
    device=device,
)

5. Set the motion planning algorithm (CL-RRT) with dynamics model based on probabilistic traversability.

In [ ]:
model_config = ModelConfig(
    mode="inference",
    inference_metric="var",
    confidence_value=0.9,
)

dynamics = UnicycleModel(
    grid_map=grid_map,
    model_config=model_config,
    device=device,
)

# Initialize the DWA local planner and the objectives
objectives = Objectives(
    dynamics=dynamics,
    goal_pos=env._goal_pos,
    stuck_threshold=env.stuck_threshold,
)

solver = CLRRT(
    dim_state=3,
    dim_control=2,
    dynamics=dynamics,
    objectives=objectives,
    grid_map=grid_map,
    delta_t=delta_t,
)

# Set directory to save the video
results_directory = os.path.join(
    base_directory,
    f"results/" f"dataset{dataset_index:02d}/" f"subset{subset_index:02d}/" "CL_RRT/",
)

# Check if the directory exists
if not os.path.exists(os.path.dirname(results_directory)):
    os.makedirs(os.path.dirname(results_directory))

In [ ]:
state = env.reset(seed)
is_collisions = env.collision_check(states=state.unsqueeze(0).unsqueeze(0))
is_replan = True
for t in range(int(time_limit / delta_t)):
    with torch.no_grad():
        if t == 0 or is_replan:
            action_seq, state_seq = solver.forward(state=state)
            action_index = 0
            is_replan = False

        if t > 0:
            pos_deviation_seq = torch.norm(state_seq[:, :, :2] - state[:2], dim=2)
            deviation = torch.min(pos_deviation_seq)
            is_replan = deviation > 1.0
            if is_replan:
                print(f"Replan at t={t}, deviation={deviation}")
                continue

        action = action_seq[action_index, :]
        action_index += 1

    state, reward, is_terminated, is_truncated = env.step(action)

    is_collisions = env.collision_check(states=state_seq)

    env.render(trajectory=state_seq, is_collisions=is_collisions, tree=solver.tree)

    if is_terminated:
        print("Goal Reached!")
        break

    if is_truncated:
        print("Time Limit Exceeded!")
        break

env.close(file_path=results_directory)

gif_file = os.path.join(results_directory, f"{instance_name}.gif")
Image(filename=gif_file)